上位チームの戦略を知ることは、シミュレーションコンペではとても重要です。過去のシミュレーションコンペ（Halite & Football）では、上位チームの対戦ログを学習したチームが金メダルを取得しています。
このnotebookでは、最上位チーム(nagiss)の対戦ログをダウンロードします。

Knowing the top team's strategy is very important in Simulation competition. In past simulation competitions (Halite & Football), the team that learned from logs of the top teams got the gold medal.
In this notebook, we download the match logs of the top team(nagiss).

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [ ]:
BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

In [ ]:
# inital team list

r = requests.post(list_url, json = {"submissionId":  18703109}) # ID is sample value
rj = r.json()

teams_df = pd.DataFrame(rj['result']['teams'])

In [ ]:
teams_df.sort_values('publicLeaderboardRank', inplace = True)
teams_df.head(10)

## Get Topteam(nagiss) Episode

In [ ]:
def saveEpisode(sub_id, epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('{}_{}.json'.format(sub_id, epid), 'w') as f:
        f.write(re.json()['result']['replay'])

    # save episode info
    with open('{}_{}_info.json'.format(sub_id, epid), 'w') as f:
        json.dump([r for r in rj['result']['episodes'] if r['id']==epid][0], f)

In [ ]:
sub = 18669376 # nagiss's public LB submission ID

start_time = datetime.datetime.now()
r = BUFFER;
result = requests.post(list_url, json = {"submissionId":  int(sub)})
team_json = result.json()
team_df = pd.DataFrame(team_json['result']['episodes'])
print('{} games for {}'.format(len(team_df), sub))

for i in range(len(team_df)):
    epid = team_df.id.iloc[i]
    
    saveEpisode(sub, epid, team_json); r+=1;
    try:
        size = os.path.getsize('{}_{}.json'.format(sub, epid)) / 1e6
        print('Saved Episode #{} @ {:.1f}MB'.format(epid, size))
    except:
        print('file {}_{}.json did not seem to save'.format(sub, epid))    
    if r > (datetime.datetime.now() - start_time).seconds:
        time.sleep( r - (datetime.datetime.now() - start_time).seconds)